# Supermarket Sales Data

Link: https://www.kaggle.com/datasets/yapwh1208/supermarket-sales-data?select=annex4.csv

This is a dataset of sales data of vegetables in a supermarket.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df1 = pd.read_csv('data/annex1.csv')
df2 = pd.read_csv('data/annex2.csv')
df3 = pd.read_csv('data/annex3.csv')
df4 = pd.read_csv('data/annex4.csv')

In [ ]:
display(df1.shape)
display(df1.head(3))

display(df2.shape)
display(df2.head(3))

display(df3.shape)
display(df3.head(3))

display(df4.shape)
display(df4.head(3))

In [ ]:
len(df2['Item Code'].unique())